In [1]:
import os
import glob

import time
import datetime

import numpy as np
import pandas as pd

from matplotlib import pyplot as plt

In [2]:
# 実行結果の日付
log_datetime = '20230402_0224'

In [4]:
RESULT_FILES =  os.path.join('/home', 'jovyan', 'work', 'result', '*.csv')

result_list = []

for file in glob.glob(RESULT_FILES):
    if log_datetime not in file:
        continue
        
    print(file)
    result_list.append(file)
    
    prog_tag = file.replace('.csv', '').split('/')[-1]
    prog_tag = "_".join(prog_tag.split('_')[2:3])
    
print()
print('Prog tag: ' + prog_tag)

/home/jovyan/work/result/20230402_0224_min-block-expand_02_testset_sys_main.csv

Prog tag: min-block-expand


In [5]:
CHAMP_DIR = os.path.join('/home', 'jovyan', 'work', 'result', 'champion')

champ_path = os.path.join(CHAMP_DIR, 'champ_all.csv')
champ_df = pd.read_csv(champ_path)
champ_df_bk = champ_df.copy()

In [6]:
def update_champion(file_name, champ_df):
    result_df = pd.read_csv(file_name)

    result_df = result_df[['seed', 'score', 'result']]

    merge_df = pd.merge(champ_df, result_df, on=['seed'], how='left')

    print('New highest record: {}'.format(sum(merge_df['champion_score'] > merge_df['score'])))
    
    new_record = merge_df['champion_score'] >= merge_df['score']

    merge_df.loc[new_record, 'champion_score'] = merge_df[new_record]['score']
    merge_df.loc[new_record, 'champion_result'] = merge_df[new_record]['result']
    merge_df.loc[new_record, 'champion'] = prog_tag

    merge_df.drop('score', axis=1, inplace=True)
    merge_df.drop('result', axis=1, inplace=True)
    
    return merge_df

In [7]:
for file in result_list:
    champ_df = update_champion(file, champ_df)

New highest record: 83


In [8]:
f_all = champ_df['champion'] == prog_tag
print('New updated record: {} problems'.format(sum(f_all)))

New updated record: 84 problems


## 結果を更新

In [9]:
t_now = datetime.datetime.now() + datetime.timedelta(hours=9)
time_str = t_now.strftime('%Y%m%d_%H%M')
    
if sum(f_all) > 0:
    champ_backup_path = champ_path.replace('.csv', '_' + time_str + '.csv')
    champ_df.to_csv(champ_backup_path, index=False)

    champ_df.to_csv(champ_path, index=False)

## 初期化

In [32]:
if False:
    # テストケースマスタ
    MASTER_PATH = os.path.join('/home', 'jovyan', 'work', '01_testset', '03_testset_stress_master.csv')
    master_df = pd.read_csv(MASTER_PATH)

    master_df['champion'] = 'undef'
    master_df['champion_score'] = 14*14*14 * 10 ** 9
    master_df['champion_result'] = 'undef'

    INIT_PATH = os.path.join('/home', 'jovyan', 'work', 'result', 'champion', 'champ_init.csv')
    master_df.to_csv(INIT_PATH, index=False)